# Working with structured data - Part 1

This notebook accompanies the script <strong><span style="color:red;">06_pandas_part_B.pdf</span></strong>  and provides practical examples related to its content.

In [ ]:
import numpy as np
import pandas as pd

<hr style="border: none; height: 20px; background-color: green;">

# 1. Hierarchical Indexing with MultiIndex

### Reminder: Indexing a **Series**
A `Series` maps **index labels** to values. Here we track the population of US states in 2000.

In [ ]:
populations = [33871648, 18976457, 20851820]
index = ['California', 'New York', 'Texas']
state_population_2000 = pd.Series(populations, index=index)
state_population_2000

### Hierarchical Indexing - The "bad/tempting" way: tuples as keys
If we track both `state` and `year`, it is tempting to use tuples as the index.

In [ ]:
index = [
    ('California', 2000), ('California', 2010),
    ('New York', 2000), ('New York', 2010),
    ('Texas', 2000), ('Texas', 2010),
]
populations = [33871648, 37253956, 
               18976457, 19378102, 
               20851820, 25145561]
ser_state_population = pd.Series(populations, index=index)
ser_state_population

Direct indexing with tuple keys works, and slicing can work if the index is sorted.

In [ ]:
ser_state_population[('California', 2010)]

In [ ]:
ser_state_population[('California', 2010):('Texas', 2010)]

#### Accessing All States for a Specific Year (2010)

This is a bit trickier because tuples are used as the index, so we need to filter the data (with Fancy indexing).

In [ ]:
ser_state_population.loc[[idx for idx in ser_state_population.index if idx[1] == 2010]]

### A cleaner way: A real `MultiIndex`
Use `pd.MultiIndex.from_tuples()` so `state` and `year` become separate index levels.

In [ ]:
index = [
    ('California', 2000), ('California', 2010),
    ('New York', 2000), ('New York', 2010),
    ('Texas', 2000), ('Texas', 2010),
]
index = pd.MultiIndex.from_tuples(index)
index


#### Setting `index.names` helps us label the hierarchy in our `MultiIndex`

In [ ]:
index.names = ['state', 'year']
index

#### Create the multi-indexed Series

In [ ]:
populations = [33871648, 37253956, 
               18976457, 19378102, 
               20851820, 25145561]
ser_state_population = pd.Series(populations, index=index)
ser_state_population

### Multi-indexed Series: Attributes

#### Levels
Shows the unique values at each level of the MultiIndex


In [ ]:
idx = ser_state_population.index
print('Levels:', idx.levels)

#### Codes
Encodes which level values correspond to each row in the MultiIndex


In [ ]:
print('Codes:', idx.codes)

#### Names (index.names)
Labels each level of the MultiIndex for better readability and operations

In [ ]:
print('Names:', idx.names)

### Selecting with `.xs()`
`.xs()` selects data across a specific index level in a MultiIndex Series or DataFrame.

In [ ]:
ser_state_population.xs(2010, level='year')

In [ ]:
ser_state_population.xs('California', level='state')

### Comparing `.loc[]`, `.xs()`, and `.iloc[]`

In [ ]:
# .loc[] (label-based)
print('California 2010:', ser_state_population.loc[('California', 2010)])
print('\nAll years for California:\n', ser_state_population.loc['California'])
print('\nAll states for 2010:\n', ser_state_population.loc[:, 2010])

In [ ]:
# .xs() (level-based)
print('\nAll states for 2010 with .xs():\n', ser_state_population.xs(2010, level='year'))
print('\nGet population for all years for California:\n', ser_state_population.xs('California', level='state'))

In [ ]:
# .iloc[] (position-based)
print('\nRow index 2:', ser_state_population.iloc[2])
print('Last row:', ser_state_population.iloc[-1])

## Reshaping with `.unstack()` and `.stack()`
`.unstack()` turns one index level into columns, adding a new dimension.    
`.stack()` reverses that operation.

In [ ]:
df_state_population = ser_state_population.unstack()
print(type(ser_state_population))
df_state_population

In [ ]:
ser_state_population = df_state_population.stack()
print(type(ser_state_population))
ser_state_population

## MultiIndex from an existing DataFrame

Often you start with a regular DataFrame and then call `.set_index()`.

In [ ]:
df_state_population = pd.read_csv('../data/csv/population_dataset.csv')
df_state_population

In [ ]:
state_population_df = df_state_population.set_index(['State', 'Year'])
state_population_df

In [ ]:
state_population_df.unstack()

### Simulating a 5D Dataset with MultiIndex in Pandas
A MultiIndex can represent higher-dimensional data in a structured way inside a 2D table.

In [ ]:
countries = ['USA', 'Germany']
years = [2000, 2010]
genders = ['Male', 'Female']
age_groups = ['0-18', '19-65', '65+']
income_groups = ['Low', 'Medium', 'High']

multi_index = pd.MultiIndex.from_product(
    [countries, years, genders, age_groups, income_groups],
    names=['Country', 'Year', 'Gender', 'Age Group', 'Income Group']
)

values = np.random.randint(1_000, 100_000, size=len(multi_index))

df_5d = pd.DataFrame({'Population': values}, index=multi_index)
df_5d.head(12)

In [ ]:
# Population of females aged 19–65 in Germany in 2000
df_5d.xs(('Germany', 2000, 'Female', '19-65'), level=['Country', 'Year', 'Gender', 'Age Group'])

In [ ]:
# Compare male vs. female population for a specific year
df_5d.xs(2010, level='Year').groupby('Gender').sum()

In [ ]:
# Unstack to turn Income Group and Age Group into columns
df_5d.unstack(['Income Group', 'Age Group'])


### Why Use Hierarchical Indexing?

MultiIndex allows us to efficiently organize structured data while keeping it in a single DataFrame and access it easily

In [ ]:
# Create MultiIndex using from_product()
states = ['California', 'New York', 'Texas']
years = [2000, 2010]
index = pd.MultiIndex.from_product([states, years], names=['State', 'Year'])

# Define economic data
gdp = [1500, 2000, 900, 1100, 1200, 1600]  # in billion USD
unemployment_rate = [5.2, 6.1, 4.5, 5.3, 6.2, 7.0]  # In %

# Create DataFrame with MultiIndex
state_economy = pd.DataFrame(
    {
        'GDP (in billion USD)': gdp,
        'Unemployment Rate (%)': unemployment_rate
    },
    index=index
)

state_economy

### Applying NumPy ufuncs to MultiIndex DataFrames

All universal functions and other Pandas operations work seamlessly with hierarchical indexes

In [ ]:
# Compute mean and standard deviation of the unemployment rate
mean_unemployment = state_economy["Unemployment Rate (%)"].mean()
std_unemployment = state_economy["Unemployment Rate (%)"].std()

# Apply NumPy ufunc to standardize (Z-Score Normalization)
state_economy["Unemployment Rate (Z-Score)"] = (
    (state_economy["Unemployment Rate (%)"] - mean_unemployment) / std_unemployment
)

state_economy

### Creating a `MultiIndex` by Passing Multiple Index Arrays

In [ ]:
df = pd.DataFrame(
    np.random.rand(4, 2),
    index=[
        ['a', 'a', 'b', 'b'],
        [1, 2, 1, 2]
    ],
    columns=['data1', 'data2']
)

df

#### Creating a `MultiIndex` from a `Dictionary` with Tuple Keys

In [ ]:
data = {
    ('California', 2000): 33871648,
    ('California', 2010): 37253956,
    ('Texas', 2000): 20851820,
    ('Texas', 2010): 25145561,
    ('New York', 2000): 18976457,
    ('New York', 2010): 19378102
}

series = pd.Series(data)

series

### `MultiIndex` for columns
Rows and columns are symmetric: both can have multiple index levels.

In [ ]:
index = pd.MultiIndex.from_product(
    [[2023, 2024], [1, 2]],
    names=['year', 'visit']
)

columns = pd.MultiIndex.from_product(
    [['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
    names=['subject', 'type']
)

# Mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10  # Scale HR values
data += 37          # Shift values to a reasonable range

# Create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)

health_data

<hr style="border: none; height: 20px; background-color: green;">

# 2. Working with Missing Data

Missing values are common in real-world datasets. Pandas supports several sentinel values:
- `None` (Python)
- `np.nan` (IEEE floating point NaN)
- `pd.NA` (Pandas nullable missing value)
- `pd.NaT` (missing timestamps)


In [ ]:
# None forces object dtype if mixed into numeric Python lists
arr_obj = np.array([1, 2, None, 4], dtype=object)
arr_obj, arr_obj.dtype

In [ ]:
df = pd.DataFrame({"A": [1, 2, None, 4]})
print(df["A"].dtype)  # A becomes float64
df

In [ ]:
# NaN lives in float arrays; inserting NaN upcasts integer data to float
series = pd.Series([1, 2, np.nan, 4])
series

In [ ]:
# None in string column → becomes object
df = pd.DataFrame({"A": ["apple", None, "banana", "cherry"]})
print(df["A"].dtype)  # A becomes object

## The 1st downside of having None and data types objects

Operations on object dtype are significantly slower

In [ ]:
# Compare performance: int vs object dtype

df_int = pd.DataFrame({"A": np.arange(1e6, dtype='int')})
%timeit df_int["A"].sum()

df_object = pd.DataFrame({"A": np.arange(1e6, dtype='object')})
%timeit df_object["A"].sum()

### Handling Missing Values in Integer Arrays

#### 1. Use a Special Placeholder Valu (e.g. `-1`)

In [ ]:
# Integer array (cannot contain NaN)
arr = np.array([1, 2, -1, 4], dtype=int)

#### 2. Convert to Float

In [ ]:
# Float array (can contain NaN)
arr = np.array([1, 2, np.nan, 4], dtype=float)

#### 3. Use `numpy.ma.MaskedArray`

In [ ]:
# Mask the third value using a masked array
arr = np.ma.masked_array(arr, mask=[0, 0, 1, 0])

#### 4. Use `pd.NA` in Pandas

In [ ]:
# Pandas nullable integer Series (supports missing values)
series = pd.Series([1, 2, pd.NA, 4], dtype=pd.Int8Dtype())

## The 2nd downside of having None and data types objects

- Unlike NaN, None is not a numerical value and cannot participate in mathematical operations
- Trying to apply aggregated functions like sum() or min() on an array containing None will raise an error

In [ ]:
# NumPy array with None (becomes object dtype → sum fails)
arr = np.array([1, 2, None, 4])

try:
    result = arr.sum()
    print("Sum:", result)

except TypeError as e:
    print("TypeError during sum:", e)

In [ ]:
# Pandas DataFrame handles None as missing value
df = pd.DataFrame({"A": [1, 2, None, 4]})
df.sum()

### The IEEE Way to Handle Missing Data: NaN (Not a Number)

In [ ]:
import struct

def float_to_ieee754(value):
    """Convert a float to its 64-bit IEEE 754 binary representation."""
    bits = struct.unpack('<Q', struct.pack('<d', value))[0]
    sign = (bits >> 63) & 0x1
    exponent = (bits >> 52) & 0x7FF
    significand = bits & 0xFFFFFFFFFFFFF  # 52-bit significand

    print(f"Value: {value}")
    print(f"Sign: {sign} | Exponent: {bin(exponent)} | Significand: {bin(significand)}")
    print(f"Full 64-bit IEEE 754 representation: {bin(bits)}\n")


float_to_ieee754(np.nan)
float_to_ieee754(np.inf)
float_to_ieee754(-np.inf)  # Negative Infinity


#### Any arithmetic operation involving `NaN` results in another `NaN`

In [ ]:
# NaN propagates through arithmetic
print(1 + np.nan)   # NaN
print(0 * np.nan)   # NaN

#### Aggregates over `NaN` values will not raise an error, but will return `NaN`

In [ ]:
# Aggregations with NaN return NaN
vals2 = np.array([1, np.nan, 3, 4])
print(vals2.sum(), vals2.min(), vals2.max())

In [ ]:
# NaN-safe aggregations
print(
    np.nansum(vals2),
    np.nanmin(vals2),
    np.nanmax(vals2)
)

### `NaN` and `None` in Pandas
Pandas automatically converts standard NumPy integer arrays to float if NaN or None is assigned


In [ ]:
# Standard integer Series → None forces conversion to float (NaN)
series = pd.Series(range(2), dtype=int)
series[0] = None
series

#### Nullable integer Series → keeps integer dtype and uses `<NA>`

In [ ]:
series = pd.Series(range(2), dtype=pd.Int16Dtype())
series[0] = None
series

## Detecting Null Values

Pandas provides built-in methods to identify missing values in a dataset

In [ ]:
series = pd.Series([1.0, np.nan, 'Hello', None])
series

In [ ]:
series.isnull()

In [ ]:
series.notnull()

In [ ]:
series[series.notnull()]

### Drop missing values: `dropna()`
`dropna()` removes entire rows or columns (not single individual cells).

In [ ]:
df = pd.DataFrame(
    [
        [1.0, np.nan, 2],
        [2.0, 3.0, 5],
        [np.nan, 4.0, 6]
    ],
    columns=[0, 1, 2]
)
df

In [ ]:
df.dropna()  # drop rows with at least one missing value

In [ ]:
df.dropna(axis=1)  # drop columns that contain missing values

#### Advanced Dropping with `dropna()`
The `dropna()` function allows more control over which rows or columns are dropped, using:
- `how='all'` → Drops only rows/columns where all values are `NaN`
- `thresh=n` → Keeps rows/columns with at least n non-null values


In [ ]:
df2 = pd.DataFrame({'A': [1.0, 2.0, np.nan], 
                    'B': [np.nan, 3.0, 4.0], 
                    'C': [2, 5, 6], 
                    'D': [np.nan, np.nan, np.nan]})
df2

In [ ]:
df2.dropna(axis=1, how='all')

In [ ]:
df2.dropna(axis=0, thresh=3)

### Filling null values `fillna()`

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list("abcde"))
data

In [ ]:
# Replace NaN / None with 0
data.fillna(0)

#### Filling null values `ffill()`, `bfill()`

In [ ]:
df2

#### Forward filling — `ffill()`
Replaces missing values with the previous valid value in the same row or column. 

In [ ]:
df2.ffill(axis=0)

#### Backward filling — `bfill()`
Does the opposite, it replaces missing values with the next valid value, ensuring that gaps are filled based on future observations


In [ ]:
df2.bfill(axis=1)

### Handling Missing Data – More Advanced Methods

In [ ]:
df = pd.DataFrame(
    {
        "A": [1.0, 2.0, np.nan, 4.0, 5.0],
        "B": [np.nan, 10.0, 15.0, 20.0, np.nan]
    }
)
df

#### Replace missing values with the mean, median or mode

In [ ]:
df.fillna(df.median())

In [ ]:
df["A"].fillna(df["A"].mean())

#### Fill multiple columns in one step using a dictionary

In [ ]:
# Filling Multiple Columns at Once
df.fillna({
    "A": df["A"].mode()[0],
    "B": df["B"].mean()
})

#### Estimate missing values using linear, quadratic, or cubic interpolation

In [ ]:
# Linear, quadratic, or cubic interpolation
df.interpolate(method="linear")  # linear, quadratic, cubic

#### KNNImputer
Uses K-Nearest Neighbors (KNNImputer) to fill missing values based on similar data points

In [ ]:
df = pd.DataFrame(
    {
        "time": range(10),
        "A": [1.0, 2.0, np.nan, 4.0, 8.0, np.nan, 20.0, 40.0, np.nan, 100.0]
    }
)
df

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

<hr style="border: none; height: 20px; background-color: green;">

# 3. Concatenating Datasets
`pd.concat()` combines Series/DataFrames along an axis:
- `axis=0`: stack vertically (add rows)
- `axis=1`: stack horizontally (add columns)

#### Concatenations: reminder from NumPy

In [ ]:
arr1 = np.array([[1, 2], [3, 4]])
arr2 = np.array([[5, 6]])

# Concatenate along axis 0 (rows)
concat_rows = np.concatenate([arr1, arr2], axis=0)
print("Concatenated along rows:\n", concat_rows)

# Concatenate along axis 1 (columns)
arr3 = np.array([[5], [6]])
concat_cols = np.concatenate([arr1, arr3], axis=1)
print("\nConcatenated along columns:\n", concat_cols)

#### Concatenations in Pandas

In [ ]:
df1 = pd.DataFrame(
    {"A": [1, 2], "B": [3, 4], "C": [5, 6]},
    index=[1, 2]
)

df2 = pd.DataFrame(
    {"A": [7, 8], "B": [9, 10]},
    index=[3, 4]
)
display(df1, df2)

#### Basic row-wise concatenation 

In [ ]:
pd.concat([df1, df2])

#### Basic column-wise Concatenation (`axis=1`)

In [ ]:
pd.concat([df1, df2], axis=1)

#### Difference Between `np.concatenate()` and `pd.concat()`

Unlike NumPy’s `np.concatenate()`, Pandas preserves indices even if they are duplicated.   
This behavior can lead to unintended consequences when concatenating DataFrames.


In [ ]:
# Make duplicate indices
df2.index = df1.index

pd.concat([df1, df2])

### Duplicate indices

Unlike NumPy’s np.concatenate(), pandas preserves indices even if they are duplicated.  

By setting verify_integrity=True, pd.concat() raises a ValueError if duplicate indices are detected in the result. This helps catch unintended index duplication early, preventing potential issues when working with index-based operations.


In [ ]:
try:
    pd.concat([df1, df2], verify_integrity=True)
except ValueError as e:
    print('ValueError:', e)

Reset indices during concatenation with `ignore_index=True`.

In [ ]:
pd.concat([df1, df2], ignore_index=True)

Add a key level to create a MultiIndex in the result with `keys=`.

In [ ]:
pd.concat([df1, df2], keys=['x', 'y'])

Keep only the intersection of columns with `join='inner'` (avoids extra NaN columns).

In [ ]:
pd.concat([df1, df2], join='inner')

## Merging and Joining Datasets
`pd.merge()` / `df.merge()` combine tables by aligning rows on key columns (SQL-style joins).

In [ ]:
df_1 = pd.DataFrame({
    'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']
})
df_2 = pd.DataFrame({
    'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
    'hire_date': [2004, 2008, 2012, 2014]
})
display(df_1, df_2)

#### `pd.merge()` vs. `df.merge()` – Two Ways to Merge DataFrames
Both `pd.merge()` and `df.merge()` perform the same operation, merging two DataFrames based on a key


In [ ]:
pd.merge(df_1, df_2)

In [ ]:
df_1.merge(df_2)

### The `on` Option in `pd.merge()`

- The `on` parameter specifies the column(s) that should be used as the key(s) for merging
- It ensures that the merge operation is performed based on a common identifier between both DataFrames
- Without `on`, pandas tries to auto-detect common columns, but it's best to explicitly define them


In [ ]:
df_1.merge(df_2, on='employee')

### Many-to-many join
If both sides have duplicate keys, the result can expand (cartesian product per key).

In [ ]:
df_3 = pd.DataFrame({
    "employee": ["Bob", "Jake", "Lisa", "Sue"],
    "group": ["Accounting", "Engineering", "Engineering", "HR"],
    "hire_date": [2008, 2012, 2004, 2014]
})

df_4 = pd.DataFrame({
    "group": ["Accounting", "Engineering", "HR"],
    "supervisor": ["Carly", "Guido", "Steve"]
})

display(df_3, df_4)

In [ ]:
pd.merge(df_3, df_4, on='group')

#### Many-to-Many Joins in pd.merge()
A Many-to-Many join occurs when both key columns in the merging DataFrames contain duplicate values


In [ ]:
df_5 = pd.DataFrame({
    "employee": ["Bob", "Jake", "Lisa", "Sue"],
    "group": ["Accounting", "Engineering", "Engineering", "HR"]
})

df_6 = pd.DataFrame({
    "group": [
        "Accounting", "Accounting",
        "Engineering", "Engineering",
        "HR", "HR"
    ],
    "skills": [
        "math", "spreadsheets",
        "coding", "linux",
        "spreadsheets", "organization"
    ]
})

In [ ]:
pd.merge(df_5, df_6, on='group')

### Different key names: `left_on` / `right_on`
Useful when the key column names differ.

In [ ]:
df_7 = pd.DataFrame({
    "name": ["Bob", "Jake", "Lisa", "Sue"],
    "salary": [70000, 80000, 120000, 90000]
})

pd.merge(df_1, df_7, left_on='employee', right_on='name')

In [ ]:
# Drop redundant columns if needed
df = pd.merge(df_1, df_7, left_on='employee', right_on='name')
df.drop('name', axis=1)

### Join strategies with `how=`
- `inner`: keep only matches
- `outer`: keep all rows from both sides
- `left`: keep all left rows
- `right`: keep all right rows

In [ ]:
df_8 = pd.DataFrame({
    "employee": ["Bob", "Jake", "Lisa"],
    "salary": [70000, 80000, 120000]
})
display(df_1, df_8)

In [ ]:
pd.merge(df_1, df_8, on='employee', how='left')

### `.join()` merges on index by default
Use `.join()` when your identifiers are already in the index.

In [ ]:
df1 = pd.DataFrame({
    "group": ["Accounting", "Engineering", "Engineering", "HR"]
}, index=["Bob", "Jake", "Lisa", "Sue"])

df2 = pd.DataFrame({
    "hire_date": [2004, 2008, 2012, 2014]
}, index=["Lisa", "Bob", "Jake", "Sue"])
display(df1, df2)

In [ ]:
df1.join(df2)